# Decision Tree Implementation - ID3

### Requirements:

#### Diturunkan Sendiri:
- Program bisa membuat sebuah objek pohon yang bisa menyimpan attributes dari tree
- Objek pohon dapat membuat decision tree dari data yang diberikan, dan menyimpan atribut-atribut dari pohon tersebut
- Objek pohon dapat menyimpan node-node yang merupakan splitting points untuk membuat keputusan
- Objek pohon dapat mengakses seluruh node yang ada pada pohon
- Objek pohon dapat memilih splitting point untuk tiap keadaan; apakah menggunakan metrik information gain atau gain ratio
- Objek pohon dapat mempertimbangkan atribut yang value-nya continuous dan diskrit
- Objek pohon dapat mempertimbangkan atribut yang mempunyai missing value
- Objek pohon dapat melakukan post-pruning dengan menggunakan 20% data untuk validasi. Detil pruning kurang lebih: https://www.quora.com/How-can-I-find-a-real-step-by-step-example-of-a-decision-tree-pruning-to-overcome-overfitting
- Objek pohon dapat menampilkan pohon yang dibuat
- Objek node dapat melakukan splitting pada dataset (menentukan keputusan harus ke node mana setelah suatu kondisi)
    - Objek node tahu harus melakukan splitting pada atribut apa
    - Objek node menyimpan splitting points pada atribut yang bersangkutan

#### Dari Spek:
- Overfitting training data dengan post pruning. Gunakanlah 20% training data untuk data validasi.
- Continuous-valued attribute: information gain dari kandidate.
- Alternative measures for selecting attributes: gain ratio.
- Handling missing attribute value: most common target value.
- full-training the data 
- menampilkan modelnya.

In [17]:
import pandas as pd
import numpy as np
import math

In [18]:
data = pd.read_csv("play_tennis.csv")
data

,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


In [75]:
new_data = data.copy()
new_data['trial'] = pd.Series([x for x in range(0,14)])
new_data.sort_values(by='trial', ascending=False)
new_data

,day,outlook,temp,humidity,wind,play,trial
0,D1,Sunny,Hot,High,Weak,No,0
1,D2,Sunny,Hot,High,Strong,No,1
2,D3,Overcast,Hot,High,Weak,Yes,2
3,D4,Rain,Mild,High,Weak,Yes,3
4,D5,Rain,Cool,Normal,Weak,Yes,4
5,D6,Rain,Cool,Normal,Strong,No,5
6,D7,Overcast,Cool,Normal,Strong,Yes,6
7,D8,Sunny,Mild,High,Weak,No,7
8,D9,Sunny,Cool,Normal,Weak,Yes,8
9,D10,Rain,Mild,Normal,Weak,Yes,9


In [76]:
#definisi kelas Node
#Node merupakan split point pada tree. 
#Kelas ini menyimpan data yang ada pada suatu split point, atribut apa yang digunakan untuk splitting, dan tipe atribut tsb. Atau jika node merupakan daun maka disimpan value-nya
#Kelas ini dapat menentukan splitting point kebawah dari suatu node, baik atribut splittingnya kontinu maupun diskrit
#Atribut-atribut Node: 
# - data: subset data
# - split_attr: nama atribut yang akan di split
# - split_values: value cabang dari node (merupakan satu integer jika continuous, dan multiple values jika categorical)
# - target_attr: atribut label/atribut target prediksi
# - attr_cont_split: splitting point dari atribut tsb (jika atribut tsb kontinu)
# - is_leaf: apakah node merupakan daun atau tidak
# - leaf_value: nilai hasil prediksi jika node merupakan daun
# - childs: anak dari node yang berupa node
class Node:
    #konstruktor
    def __init__(self, data, split_attr, target_attr, is_continuous=False, split_value_continuous=None):
        self.data = data
        self.split_attr = split_attr
        self.target_attr = target_attr
        self.childs = []
        self.is_leaf = False
        self.split_values = [split_value_continuous]

    #check apakah split attribute == numerik
    def is_attr_categorical(self):
        return self.data[self.split_attr].dtype == 'O'
        
    #check apakah node == daun
    def check_if_leaf(self):
        return self.data[self.target_attr].nunique() == 1
        
    #buat node menjadi daun
    def make_leaf(self):
        if(self.check_if_leaf()):
            self.is_leaf = True
            self.leaf_value = self.data[self.target_attr].unique()[0]
            
    #cari split-split yang memungkinkan pada atribut continuous
    def find_possible_splits_continuous(self, sorted_data):
        sorted_target = sorted_data[self.target_attr].values.tolist()
        sorted_attr = sorted_data[self.split_attr].values.tolist()
        prev_target_value = sorted_target[0]
        possible_splits = []
        #iterasi target value, cari titik-titik dimana 
        try:
            for i in range(1, len(sorted_target)):
                el = sorted_target[i]
                if (prev_target_value != el):
                    possible_splits.append(0.5*(sorted_attr[i] + sorted_attr[i-1]))
                prev_target_value = el
        except Exception as e:
            print(e)
        finally:
            return possible_splits
    
    #get splits node jika node bukan daun
    def get_splits(self):
        if( not self.check_if_leaf()):
            #jika atribut split categorical
            if(self.is_attr_categorical()):
                #tentukan split values
                self.split_values = self.data[self.split_attr].unique()
            #jika atribut numerik / continuous, split value sudah didefinisikan sejak konstruksi objek
            return self.split_values
                        
    #add a child to a node
    def add_child(self, node):
        self.childs.append(node)

In [56]:
#definisi kelas Tree
#Kelas ini mengkonstruksi decision tree dengan menghubungkan sekumpulan node, juga memilih untuk tiap node 
#atribut apa yang akan digunakan untuk splitting. Kelas ini dapat mempertimbangkan atribut yang mengandung nilai null.
#Metrik yang  digunakan bisa dipilih antara information gain atau gain ratio.
#Kelas ini dapat melakukan pruning pada tree yang dibuat, dan juga dapat mencetak model tree yang telah dibuat
#Atribut-atribut Tree:
# - data: merupakan data yang digunakan untuk training
# - target_attr: atribut yang menjadi target prediksi (label)
# - root: node yang merupakan root
class Tree:
    #konstruktor
    def __init__(self, data, target_attr):
        self.data = data
        self.target_attr = target_attr
        self.root = None
    
    #cari entropi total pada data
    def total_entropy(self, data):
        proportion = self.data[self.target_attr].value_counts()/len(data)
        entropy = 0
        for p in proportion.tolist():
            entropy -= p*math.log(p,2)
        return entropy
    
    #hitung information gain dari suatu kolom
    def info_gain(self, data, kolom):
        data_entropy = self.total_entropy(data)
        proportion_kolom = data[kolom].value_counts()/len(data)
        sum_entropy_kolom = 0
        for value_kolom, value_proportion in zip(proportion_kolom.index.tolist(), proportion_kolom.tolist()):
            entropy_value_kolom = total_entropy(data[data[kolom] == value_kolom])
            sum_entropy_kolom -= value_proportion*entropy_value_kolom
            print('value entropy kolom for', kolom, ':', value_kolom, ':', value_proportion, '=', entropy_value_kolom )
        print('sum entropy kolom for', kolom, '=', sum_entropy_kolom)
        return data_entropy + sum_entropy_kolom
    
    #cari split-split yang memungkinkan pada atribut continuous
    def find_possible_splits_continuous(self, sorted_data):
        sorted_target = sorted_data[self.target_attr].values.tolist()
        sorted_attr = sorted_data[self.split_attr].values.tolist()
        prev_target_value = sorted_target[0]
        possible_splits = []
        #iterasi target value, cari titik-titik dimana 
        try:
            for i in range(1, len(sorted_target)):
                el = sorted_target[i]
                if (prev_target_value != el):
                    possible_splits.append(0.5*(sorted_attr[i] + sorted_attr[i-1]))
                prev_target_value = el
        except Exception as e:
            print(e)
        finally:
            return possible_splits
    
    #cari gain dari tiap split dan cari split optimum
    def find_optimum_split_continuous(pos_splits, sorted_data):
        optimum_split = 0
        max_info_gain = -1
        #iterate split
        for i, el in enumerate(pos_splits):
            #hitung information gain
            current_gain = calculate_info_gain_continuous(el, sorted_data)
            #jika information gain lebih dari sebelumnya, ganti optimum split
            if(current_gain > max_info_gain):
                max_info_gain = current_gain
                optimum_split = el
        return optimum_split
    
    #cari information gain pada suatu split continuous
    def calculate_info_gain_continuous(split_value, sorted_data):
        data_entropy = self.total_entropy(sorted_data)
        #pisah data mjd "<=" dan ">" split_value
        data_less_than_equal = sorted_data.loc[sorted_data[self.split_attr] <= split_value]
        data_more_than = sorted_data.loc[sorted_data[self.split_attr] > split_value]
        #hitung entropi kolom
        entropy_less_than_equal = (len(data_less_than_equal)/len(sorted_data)) * total_entropy(data_less_than_equal)
        entropy_more_than = (len(data_more_than)/len(sorted_data)) * total_entropy(data_more_than)
        return data_entropy - entropy_less_than_equal - entropy_more_than
    
    #check apakah attribute == numerik
    def is_attr_categorical(self, attr):
        return self.data[attr].dtype == 'O'
    
    #buat tree
    def make_tree(self):
        #cari info_gain dari masing-masing kolom 
        data_X = self.data.drop(self.target_attr, axis=1)
        max_info_gain = -1
        split_attr = ""
        is_split_attr_categorical = True
        for attr in data_X.columns:
            #Jika kolom kategorikal
            if(self.is_attr_categorical(attr)):
                current_info_gain = self.info_gain(data_X, attr)
            #jika kolom numerik
            else:
                #sort data
                sorted_data = self.data.sort_values(by=attr)
                #cari split-split yang memungkinkan 
                pos_splits = self.find_possible_splits_continuous(sorted_data)
                #hitung gain dari tiap continuous split dan cari nilai optimum
                split_value_continuous = self.find_optimum_split_continuous(pos_splits, sorted_data)
                #hitung gain ketika sudah diketahui nilai optimum
                current_info_gain = self.calculate_info_gain_continuous(split_value_continuous, sorted_data)
            
            #jika ditemukan maximum info gain di kolom tertentu
            if(current_info_gain > max_info_gain):
                max_info_gain = current_info_gain
                split_attr = attr
                is_split_attr_categorical = self.is_attr_categorical(attr)
                if (not is_split_attr_categorical):
                    split_value_attr = split_value_continuous
                    
        #buat node
        #jika atribut terpilih == kategorikal
        if(is_split_attr_categorical):
            split_values = self.data[split_attr].unique()
            #iterate all split values
            for split_value in split_values:
                filtered_data = self.data[self.data[split_attr] == split_value]
                node = Node(filtered_data, split_attr, self.target_attr)
                #basis - jika node merupakan leaf
                if(node.check_if_leaf()):
                    return node
                #rekurens - jika node bukan leaf
                else
                    node.add_child(Tree(filtered_data, self.target_attr).make_tree())
                    return node
        #jika atribut terpilih == numerik & kontinu
        else:
            #filter <=
            filtered_data = self.data[self.data[self.]]
            #filter >
            
                    
        
                
        

In [71]:
#hitung entropi total dataset
def total_entropy(data):
    proportion = data['play'].value_counts()/len(data)
    entropy = 0
    for p in proportion.tolist():
        entropy -= p*math.log(p,2)
    return entropy

#hitung information gain dari suatu kolom
def gain(data, kolom):
    data_entropy = total_entropy(data)
    print('KOLOM:', kolom.upper())
    print('total entropy of current data', '=',data_entropy)
    proportion_kolom = data[kolom].value_counts()/len(data)
    sum_entropy_kolom = 0
    for value_kolom, value_proportion in zip(proportion_kolom.index.tolist(), proportion_kolom.tolist()):
        entropy_value_kolom = total_entropy(data[data[kolom] == value_kolom])
        sum_entropy_kolom -= value_proportion*entropy_value_kolom
        print('value entropy kolom for', kolom, ':', value_kolom, ':', value_proportion, '=', entropy_value_kolom )
    print('sum entropy kolom for', kolom, '=', sum_entropy_kolom)
    return data_entropy + sum_entropy_kolom

#get current_data
def get_node_data(data, kolom, value):
    new_data = data[data[kolom] == value]
    return new_data.drop(kolom, axis=1)

#get current_columns
def get_current_columns(data):
    return data.drop('play', axis=1).columns

### Iterasi 1

In [72]:
current_data = data
current_gains = []
for kolom in get_current_columns(current_data):
    gain_kolom = gain(current_data, kolom)
    print("gain", kolom, ":", gain_kolom)
    current_gains.append([kolom, gain_kolom])
current_gains = pd.Series([x[1] for x in current_gains], index=[x[0] for x in current_gains])
print('GAINS\n', current_gains)
print('AKTIVITAS TERKAIT', current_data['play'].value_counts().index.tolist())

KOLOM: DAY
total entropy of current data = 0.9402859586706309
value entropy kolom for day : D7 : 0.07142857142857142 = 0.0
value entropy kolom for day : D8 : 0.07142857142857142 = 0.0
value entropy kolom for day : D12 : 0.07142857142857142 = 0.0
value entropy kolom for day : D11 : 0.07142857142857142 = 0.0
value entropy kolom for day : D6 : 0.07142857142857142 = 0.0
value entropy kolom for day : D2 : 0.07142857142857142 = 0.0
value entropy kolom for day : D3 : 0.07142857142857142 = 0.0
value entropy kolom for day : D14 : 0.07142857142857142 = 0.0
value entropy kolom for day : D5 : 0.07142857142857142 = 0.0
value entropy kolom for day : D9 : 0.07142857142857142 = 0.0
value entropy kolom for day : D13 : 0.07142857142857142 = 0.0
value entropy kolom for day : D10 : 0.07142857142857142 = 0.0
value entropy kolom for day : D4 : 0.07142857142857142 = 0.0
value entropy kolom for day : D1 : 0.07142857142857142 = 0.0
sum entropy kolom for day = 0.0
gain day : 0.9402859586706309
KOLOM: OUTLOOK
to

In [36]:
current_data

,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


#### Root = Ada Hangout

### Iterasi 2

#### ada hangout = ya

In [73]:
current_data = get_node_data(data, 'ada hangout', 'ya')
current_gains = []
for kolom in get_current_columns(current_data):
    gain_kolom = gain(current_data, kolom)
    print("gain", kolom, ":", gain_kolom)
    current_gains.append([kolom, gain_kolom])
current_gains = pd.Series([x[1] for x in current_gains], index=[x[0] for x in current_gains])
print('GAINS\n', current_gains)
print('AKTIVITAS TERKAIT', current_data['aktivitas'].value_counts().index.tolist())

KeyError: 'ada hangout'

In [ ]:
current_data

#### ada hangout = tidak

In [38]:
current_data = get_node_data(data, 'ada hangout', 'tidak')
current_gains = []
for kolom in get_current_columns(current_data):
    gain_kolom = gain(current_data, kolom)
    print("gain", kolom, ":", gain_kolom)
    current_gains.append([kolom, gain_kolom])
current_gains = pd.Series([x[1] for x in current_gains], index=[x[0] for x in current_gains])
print('GAINS\n', current_gains)
print('AKTIVITAS TERKAIT', current_data['aktivitas'].value_counts().index.tolist())

KeyError: 'ada hangout'

In [39]:
current_data

,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


### Iterasi 3

#### ada hangout = tidak ^ deadline = urgent

In [40]:
current_data = get_node_data(get_node_data(data, 'ada hangout', 'tidak'), 'deadline', 'urgent')
current_gains = []
for kolom in get_current_columns(current_data):
    gain_kolom = gain(current_data, kolom)
    print("gain", kolom, ":", gain_kolom)
    current_gains.append([kolom, gain_kolom])
current_gains = pd.Series([x[1] for x in current_gains], index=[x[0] for x in current_gains])
print('GAINS\n', current_gains)
print('AKTIVITAS TERKAIT', current_data['aktivitas'].value_counts().index.tolist())

KeyError: 'ada hangout'

In [41]:
current_data

,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


#### ada hangout = tidak ^ deadline = dekat

In [42]:
current_data = get_node_data(get_node_data(data, 'ada hangout', 'tidak'), 'deadline', 'dekat')
current_gains = []
for kolom in get_current_columns(current_data):
    gain_kolom = gain(current_data, kolom)
    print("gain", kolom, ":", gain_kolom)
    current_gains.append([kolom, gain_kolom])
current_gains = pd.Series([x[1] for x in current_gains], index=[x[0] for x in current_gains])
print('GAINS\n', current_gains)
print('AKTIVITAS TERKAIT', current_data['aktivitas'].value_counts().index.tolist())

KeyError: 'ada hangout'

In [43]:
current_data

,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


#### ada hangout = tidak ^ deadline = tidak ada

In [44]:
current_data = get_node_data(get_node_data(data, 'ada hangout', 'tidak'), 'deadline', 'tidak ada')
current_gains = []
for kolom in get_current_columns(current_data):
    gain_kolom = gain(current_data, kolom)
    print("gain", kolom, ":", gain_kolom)
    current_gains.append([kolom, gain_kolom])
current_gains = pd.Series([x[1] for x in current_gains], index=[x[0] for x in current_gains])
print('GAINS\n', current_gains)
print('AKTIVITAS TERKAIT', current_data['aktivitas'].value_counts().index.tolist())

KeyError: 'ada hangout'

In [45]:
current_data

,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


### Iterasi 4

#### ada hangout = tidak ^ deadline = dekat ^ malas = tidak

In [16]:
current_data = get_node_data(get_node_data(get_node_data(data, 'ada hangout', 'tidak'), 'deadline', 'dekat'), 'malas', 'tidak')
current_gains = []
for kolom in get_current_columns(current_data):
    gain_kolom = gain(current_data, kolom)
    print("gain", kolom, ":", gain_kolom)
    current_gains.append([kolom, gain_kolom])
current_gains = pd.Series([x[1] for x in current_gains], index=[x[0] for x in current_gains])
print('GAINS\n', current_gains)
print('AKTIVITAS TERKAIT', current_data['aktivitas'].value_counts().index.tolist())

GAINS
 Series([], dtype: float64)
AKTIVITAS TERKAIT ['belajar']


In [17]:
current_data

,aktivitas
6,belajar


#### ada hangout = tidak ^ deadline = dekat ^ malas = ya

In [18]:
current_data = get_node_data(get_node_data(get_node_data(data, 'ada hangout', 'tidak'), 'deadline', 'dekat'), 'malas', 'ya')
current_gains = []
for kolom in get_current_columns(current_data):
    gain_kolom = gain(current_data, kolom)
    print("gain", kolom, ":", gain_kolom)
    current_gains.append([kolom, gain_kolom])
current_gains = pd.Series([x[1] for x in current_gains], index=[x[0] for x in current_gains])
print('GAINS\n', current_gains)
print('AKTIVITAS TERKAIT', current_data['aktivitas'].value_counts().index.tolist())

GAINS
 Series([], dtype: float64)
AKTIVITAS TERKAIT ['nonton']


In [19]:
current_data

,aktivitas
7,nonton
